In [1]:
import os
import pandas as pd
import numpy as np
from src.utils import load_json, save_json
from omegaconf import OmegaConf
from collections import defaultdict
from pathlib import Path
filepaths = OmegaConf.load("../configs/filepaths/base.yaml")

# Fold reaction directions into single entries and save known reactions

In [6]:
# Fold reaction directions into single entry

'''
Assumes:
- Enzymes of rxn and its reverse are identical
- No more than one min rule (& reaction center) mapped to a reaction
- The reverse reverse smarts are the same as the smarts of a reaction
- Smarts are aligned with the mapped template and reaction center tuples

Stores folded as:
- Smarts & key from the smaller key arbitrarily chosen as smarts and key of folded
- Keeps a tuple of reaction center tuple tuples
- Only takes reactions that were mapped to minimal rules
'''
krs = load_json("./data/sprhea/sprhea_240310_v3_min_mapped.json")
krs = {int(k): krs[k] for k in krs}
dest = "./data/sprhea/v3_folded_pt.json"

# Enzyme filtering criteria
whitelist = [
    'Evidence at protein level',
    'Evidence at transcript level',
    # 'Inferred from homology',
]

direction_pairs = [tuple(sorted((k, krs[k]['reverse']))) for k in krs]
direction_pairs = set(direction_pairs)

krs_folded = {}
for fwd, rev in direction_pairs:
    # Skip unmapped
    if krs[fwd]['min_rule'] is None or krs[rev]['min_rule'] is None:
        continue
    
    enz = [e for e in krs[fwd]['enzymes'] if e['existence'] in whitelist]

    # Skip enzymeless
    if len(enz) == 0:
        print(fwd, rev)
        continue
    
    krs_folded[fwd] = {
        'smarts': krs[fwd]['smarts'],
        'min_rules': [krs[fwd]['min_rule'], krs[rev]['min_rule']],
        'rcs': [krs[fwd]['reaction_center'],  krs[rev]['reaction_center']],
        'enzymes': enz,
        'rhea_ids': list(set(krs[fwd]['rhea_ids'] + krs[rev]['rhea_ids'])),
        'reverse': rev
    }

print(len(krs_folded))
save_json(krs_folded, dest)

11589 12691
2374 14165
7070 11985
15440 18344
3717 11316
10133 13848
8958 14268
8789 19724
13506 17190
18138 19187
3095 16983
397 12178
6449 18821
6236 7359
5694 6997
15918 16980
4366 11273
12040 14678
3826 18972
18879 19569
18995 19249
8313 8739
14685 18539
3295 4478
15331 19510
17208 18942
17198 18285
10027 11271
17736 18150
12065 20237
17856 20447
18131 18228
12826 16422
11994 16510
5765 11204
9529 12707
15070 17869
19052 19905
11884 14331
19665 20109
13231 13567
17380 17770
13560 16779
6122 13613
10927 16236
11818 17521
15126 19224
9280 10733
8603 15607
10337 20080
993 10174
3235 17855
7114 15645
3106 4628
1821 10587
6598


In [7]:
# Save toc to file
save_to = "./data/sprhea/v3_folded_pt.csv"

# Filter out duplicate sequences

# Seq to upid dict
unique_seqs = defaultdict(set)
for k,v in krs_folded.items():
    for e in v['enzymes']:
        unique_seqs[e['sequence']].add(e['uniprot_id'])

for k,v in unique_seqs.items():
    unique_seqs[k] = sorted(v)

# Make toc csv thing
chosen_upid_to_rhashes = defaultdict(set)
chosen_upid_to_sequence = {}
for k,v in krs_folded.items():
    for e in v['enzymes']:
        seq = e['sequence']
        chosen_upid = unique_seqs[seq][0] # Choose first of sorted upids of unique seq
        chosen_upid_to_rhashes[chosen_upid].add(str(k))
        chosen_upid_to_sequence[chosen_upid] = seq

data = {'Entry':[], 'Label':[], 'Sequence':[]}
for id in chosen_upid_to_sequence.keys():
     data['Entry'].append(id)
     data['Label'].append(";".join(chosen_upid_to_rhashes[id]))
     data['Sequence'].append(chosen_upid_to_sequence[id])

# Save to sp_ops subdir
df = pd.DataFrame(data=data)
df.to_csv(save_to, index=False, sep='\t')
print(len(df))
df.head()


26127


,Entry,Label,Sequence
0,P0A6W3,1123,MLVWLAEHLVKYYSGFNVFSYLTFRAIVSLLTALFISLWMGPRMIA...
1,P9WMW7,1123,MRQILIAVAVAVTVSILLTPVLIRLFTKQGFGHQIREDGPPSHHTK...
2,O66465,1123,MLYQLALLLKDYWFAFNVLKYITFRSFTAVLIAFFLTLVLSPSFIN...
3,Q8MJ30,3097;7946,MAAAAAGEARRVLVYGGRGALGSRCVQAFRARNWWVASIDVVENEE...
4,P38489,3097;7946,MDIISVALKRHSTKAFDASKKLTPEQAEQIKTLLQYSPSSTNSQPW...


# Consolidate swissprot esm embeddings

In [8]:
import os
from tqdm import tqdm
import subprocess
import csv
from src.utils import load_embed

In [13]:
in_dir = "/projects/p30041/spn1560/hiec/data/sprhea/esm_240524"
out_dir = "/projects/p30041/spn1560/hiec/data/sprhea/esm"

mismatches = []
for fn in tqdm(os.listdir(in_dir)):
    if fn in os.listdir(out_dir):
        in_id, in_embed = load_embed(f"{in_dir}/{fn}", embed_key=33)
        out_id, out_embed = load_embed(f"{out_dir}/{fn}", embed_key=33)
        
        if all(in_embed == out_embed):
            continue
        elif all((in_embed - out_embed) < 1e-5):
            subprocess.run(['cp', f"{in_dir}/{fn}", f"{out_dir}/"])
        else:
            mismatches.append((in_id, out_id))
    else:
        subprocess.run(['cp', f"{in_dir}/{fn}", f"{out_dir}/"])
            
            


100%|██████████| 3660/3660 [01:11<00:00, 50.83it/s]


In [14]:
for fn in tqdm(os.listdir(in_dir)):
    if fn not in os.listdir(out_dir):
        in_id, in_embed = load_embed(f"{in_dir}/{fn}", embed_key=33)
        out_id, out_embed = load_embed(f"{out_dir}/{fn}", embed_key=33)
        print(in_id)

100%|██████████| 3660/3660 [00:34<00:00, 104.62it/s]


In [27]:
'''
TODO: Repeat ^^ for swissprot (from CLEAN paper)
'''

False

# Get incremental esm embeddings into project dir

In [4]:
# Put remaining uniprot ids into a fasta file
fasta_name = f"./data/sprhea/v3_incremental.fasta"
toc_path = "./data/sprhea/v3_folded_pt_ns.csv"
existing_path = "/projects/p30041/spn1560/hiec/data/sprhea/esm"

toc = pd.read_csv(toc_path, sep='\t')
all_upids = set(toc['Entry'].tolist())
toc.set_index('Entry', inplace=True)
toc.head()

existing = set([elt[:-3] for elt in os.listdir(existing_path)])
incremental = all_upids - existing
print(len(incremental))

with open(fasta_name, 'w') as f:
    for up in incremental:
        f.write('>' + up + '\n')
        f.write(toc.loc[up, 'Sequence'] + '\n')


2


# Look at how many enyzmes per reaction, reactions per enzymes in sprhea folded

In [6]:
import scipy as sp
import numpy as np
import pandas as pd
from collections import Counter
def construct_sparse_adj_mat(ds_name, toc):
        '''
        Returns sparse representation of sample x feature adjacency matrix
        and lookup of sample names from row idx key.

        Args
            - ds_name: Str name of dataset
            - toc: Table of contents csv

        Returns
            -
        '''      
        # Load from dataset "table of contents csv"
        df = pd.read_csv(f"./data/{ds_name}/{toc}.csv", delimiter='\t')
        df.set_index('Entry', inplace=True)
        sample_idx = {}
        feature_idx = {}
        
        # Construct ground truth protein-function matrix
        print(f"Constructing {ds_name}:{toc} sparse adjacency matrix")
        row, col, data = [], [], [] # For csr
        for i, elt in enumerate(df.index):
            labels = df.loc[elt, 'Label'].split(';')
            sample_idx[elt] = i
            for label in labels:
                if label in feature_idx:
                    j = feature_idx[label]
                else:
                    j = len(feature_idx)
                    feature_idx[label] = j
                row.append(i)
                col.append(j)
                data.append(1)
                
            print(f"{i}", end='\r')

        adj = sp.sparse.csr_matrix((data, (row, col)), shape=(len(sample_idx), len(feature_idx)))
        idx_sample = {v:k for k,v in sample_idx.items()}
        idx_feature = {v:k for k,v in feature_idx.items()}
            
        return adj, idx_sample, idx_feature

Constructing sprhea:sp_folded_pth sparse adjacency matrix
117170 enzymes
7092 reactions
22.375634517766496 proteins per reaction
1.3543398480839806 reactions per protein
Constructing sprhea:sp_folded_pt sparse adjacency matrix
27833

27834 enzymes
7056 reactions
7.758219954648526 proteins per reaction
1.9667313357763887 reactions per protein


In [ ]:
dataset = 'sprhea'
toc = 'sp_folded_pth'
adj, idx_sample, idx_feature = construct_sparse_adj_mat(dataset, toc)
print(adj.shape[0], 'enzymes')
print(adj.shape[1], 'reactions')
print(adj.sum(axis=0).mean(), "proteins per reaction")
print(adj.sum(axis=1).mean(), "reactions per protein")

labels_per_prot = np.array(adj.sum(axis=1)).reshape(-1,)
ct = Counter(labels_per_prot)
x, h = list(ct.keys()), list(ct.values())
plt.bar(x, h)
plt.yscale('log')
plt.xticks(np.arange(1, max(x) + 1))
plt.ylabel("# enzymes")
plt.xlabel("# reactions")
plt.xticks(rotation=90)
plt.show()

prots_per_label = np.array(adj.sum(axis=0)).reshape(-1,)
ct = Counter(prots_per_label)
x, h = list(ct.keys()), list(ct.values())
plt.figure(figsize=(9,6))
plt.bar(x, h)
plt.yscale('log')
plt.xticks(np.arange(1, max(x) + 1))
plt.ylabel("# reactions")
plt.xlabel("# enzymes")
plt.xticks(rotation=90)
plt.xscale('log')
plt.show()

In [ ]:
dataset = 'sprhea'
toc = 'sp_folded_pt'
adj, idx_sample, idx_feature = construct_sparse_adj_mat(dataset, toc)
print(adj.shape[0], 'enzymes')
print(adj.shape[1], 'reactions')
print(adj.sum(axis=0).mean(), "proteins per reaction")
print(adj.sum(axis=1).mean(), "reactions per protein")

labels_per_prot = np.array(adj.sum(axis=1)).reshape(-1,)
ct = Counter(labels_per_prot)
x, h = list(ct.keys()), list(ct.values())
plt.bar(x, h)
plt.yscale('log')
plt.xticks(np.arange(1, max(x) + 1))
plt.ylabel("# enzymes")
plt.xlabel("# reactions")
plt.xticks(rotation=90)
plt.show()

prots_per_label = np.array(adj.sum(axis=0)).reshape(-1,)
ct = Counter(prots_per_label)
x, h = list(ct.keys()), list(ct.values())
plt.figure(figsize=(9,6))
plt.bar(x, h)
plt.yscale('log')
plt.xticks(np.arange(1, max(x) + 1))
plt.ylabel("# reactions")
plt.xlabel("# enzymes")
plt.xticks(rotation=90)
plt.xscale('log')
plt.show()

# Make folded reactions x min ops labels tocs

In [1]:
from src.utils import load_known_rxns
import pandas as pd

In [2]:
krs = load_known_rxns("./data/sprhea/known_rxns_240310_v2_folded_protein_transcript.json")
save_to = "./data/sprhea/sp_folded_pt_rxns_x_min_ops.csv"

FileNotFoundError: [Errno 2] No such file or directory: './data/sprhea/known_rxns_240310_v2_folded_protein_transcript.json'

In [ ]:
# Make sure all entreis have exactly two min ops
for k,v in krs.items():
    if len(v['min_rules']) != 2:
        print(v)
        break

In [ ]:
# Make toc
data = {'Entry':[], 'Label':[]}
for k,v in krs.items():
     min_ops = sorted(v['min_rules']) # Canonical order
     data['Entry'].append(k)
     data['Label'].append("_".join(min_ops))

# Save to sp_ops subdir
df = pd.DataFrame(data=data)
df.to_csv(save_to, index=False, sep='\t')
df.head()

,Entry,Label
0,R84126efbb9122177f50f81fcff58422c31a5cf7ef7254...,rule0310_rule0311
1,R922e16ad056d0d9e87f61239c734efcf9fafb99c77cc3...,rule0120_rule0121
2,R8d9c400039c4568d063fa0a51b012da463f56d8a598cf...,rule0010_rule0011
3,R5386d7c400c7c774dc0d52a933c7b4f676c4effc8a72b...,rule0142_rule0143
4,R57b2bc5bcccd3d7e9a71b4c948ad01d180e83e03cd89e...,rule0142_rule0143


In [18]:
len(set(df.loc[:, 'Label'])), len(df)

(434, 7094)

# Make tiny test datasets for v3

In [2]:
v3_pt_ns = load_json(Path(filepaths['data']) / 'sprhea' / 'v3_folded_pt_ns.json')
toc = pd.read_csv(Path(filepaths['data']) / 'sprhea' / 'v3_folded_pt_ns.csv', sep='\t')
toc.head(10)

,Entry,Label,Sequence
0,P0A6W3,1123,MLVWLAEHLVKYYSGFNVFSYLTFRAIVSLLTALFISLWMGPRMIA...
1,P9WMW7,1123,MRQILIAVAVAVTVSILLTPVLIRLFTKQGFGHQIREDGPPSHHTK...
2,O66465,1123,MLYQLALLLKDYWFAFNVLKYITFRSFTAVLIAFFLTLVLSPSFIN...
3,Q8MJ30,3097;7946,MAAAAAGEARRVLVYGGRGALGSRCVQAFRARNWWVASIDVVENEE...
4,P38489,3097;7946,MDIISVALKRHSTKAFDASKKLTPEQAEQIKTLLQYSPSSTNSQPW...
5,P09417,3097;7946,MAAAAAAGEARRVLVYGGRGALGSRCVQAFRARNWWVASVDVVENE...
6,Q3T0Z7,3097;7946,MAAAAGEARRVLVYGGRGALGSRCVQAFRARNWWVASIDVQENEEA...
7,Q8BVI4,3097;7946,MAASGEARRVLVYGGRGALGSRCVQAFRARNWWVASIDVVENEEAS...
8,Q86A17,3097;7946,MSKNILVLGGSGALGAEVVKFFKSKSWNTISIDFRENPNADHSFTI...
9,P11348,3097;7946,MAASGEARRVLVYGGRGALGSRCVQAFRARNWWVASIDVVENEEAS...


In [3]:
len(v3_pt_ns), len(toc)

(6460, 24523)

In [4]:
k = 10_000
rng = np.random.default_rng(seed=1234)
rnd_idxs = rng.choice(toc.index, size=(k,), replace=False)
sub_toc = toc.loc[rnd_idxs]
sub_toc.head(10)

,Entry,Label,Sequence
922,Q12003,210;85,MISIVLELFQNLCCCRGFSDATIRVNDKRYRIQRLLGEGGMSFVYL...
10735,O46516,1328;2982,MAGWSCLVTGAGGFLGQRIVRLLVEEKEVQEIRALDKVFRPELREE...
1399,P36897,210;85,MEAAVAAPRPRLLLLVLAAAAAAAAALLPGATALQCFCHLCTKDNF...
638,Q9N0F1,549,MLSRSRCVSRAFSRSLSAFQKGNCPLGRRSLPGISLCQGPGYPDSR...
22881,A7MCT6,2609,MAVPPSAPVPCSPFYLRRQEPCPQCSWSMEEKAVASAGCWEPPGPP...
20969,Q5ZX22,166,MILCIDVGNSHIYGGVFDGDEIKLRFRHTSKVSTSDELGIFLKSVL...
18353,P54756,479,MRGSGPRGAGRRRPPSGGGDTPITPASLAGCYSAPRRAPLWTCLLL...
16714,Q88CC1,6126,MPANDFVSPDSIRAQFSAAMSLMYKQEVPLYGTLLELVSEINQQVM...
17402,Q9SZ92,1845,MAVGIFGLIPSSSPDELRKILQALSTKWGDVVEDFESLEVKPMKGA...
10943,Q9SA22,337;245,MGLCHSKIDKTTRKETGATSTATTTVERQSSGRLRRPRDLYSGGEI...


In [5]:
sub_krs = {}
for i, row in sub_toc.iterrows():
    for lab in row['Label'].split(';'):
        sub_krs[lab] = v3_pt_ns[lab]

In [6]:
save_json(sub_krs, Path(filepaths['data']) / 'sprhea' / f"v3_folded_n_{k}.json")
sub_toc.to_csv(Path(filepaths['data']) / 'sprhea' / f"v3_folded_n_{k}.csv", sep='\t', index=False)

# Prep cd-hit fasta

In [2]:
# Put remaining uniprot ids into a fasta file
fasta_name = Path(filepaths['data']) / 'sprhea' / "v3_folded_test.fasta"
toc_path = Path(filepaths['data']) / 'sprhea' / "v3_folded_test.csv"

toc = pd.read_csv(toc_path, sep='\t')
all_upids = set(toc['Entry'].tolist())
toc.set_index('Entry', inplace=True)
toc.head()

with open(fasta_name, 'w') as f:
    for upid, row in toc.iterrows():
        f.write('>' + upid + '\n')
        f.write(row['Sequence'] + '\n')
